In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [1]:
from rdkit.Chem import AllChem, Descriptors
from rdkit import Chem
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import sys
import pandas as pd
import numpy as np
from rdkit.Chem import MACCSkeys
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem import RDKFingerprint
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.AtomPairs import Torsions
from rdkit.Chem.Fingerprints import FingerprintMols
import os
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [2]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
m1 = Chem.MolFromSmiles('CCCC')
ds = calc.CalcDescriptors(m1)
# ds = np.asarray(list(ds))
# descriptors.append(ds)

In [3]:
a = '010101'
b = '0011'
a+b

'0101010011'

In [4]:
# np.asarray(list(ds))

In [5]:
descriptors = []
df = pd.read_csv('../dataset/two_paper_plus_transformer_combined/inhib_data/1A1_inh.csv',index_col=0)
smiles_list = df['Canonical_Smiles'].tolist()
# for i in tqdm_notebook(range(len(smiles))):
#     m1 = Chem.MolFromSmiles(smiles[i])
#     ds = calc.CalcDescriptors(m1)
#     ds = ''.join(map(str, ds))
# #     ds = ''.join(str(ds))
# #     ds = np.asarray(list(ds))
#     descriptors.append(ds)

In [15]:
descriptors[0]

'10.935818487735876-1.246205352768679410.9358184877358760.15496971319391940.8615348244596426289.158275.046288.031999667999979800.3469527260260042-0.478304313837370040.478304313837370040.34695272602600421.38888888888888881.88888888888888882.27777777777777772.0716222773934136465.629630102694213.5520417638777910.11349724958081511.6253551416177258.2219806648719375.6244628617476826.3803918077661375.0043024456352456.5466892813606522.99648002975748633.8319113198418461.681379082730112.2419620979937784-0.930000000000000211444.66858188419213.5004725305788334.3186928518744542.7279289663210857115.6560003022549.84339034864075510.0828659467693465.6010508109836880.00.05.9693052879518490.04.7945371840718220.023.2018797804650312.1327341369232237.9644816509274755.9179060461613930.014.63792753271257629.1711850684168820.00.00.036.120606992937770.029.8289197655434360.05.7495118332839054.7368629538000490.05.74951183328390523.2018797804650321.0102376072616834.7945371840718220.031.74965356016564424.2654682738

In [4]:
type(ds)

tuple

In [6]:
min(ds)

-0.06539244288085166

In [7]:
a = ds

In [8]:
[(float(i)-min(a))/(max(a)-min(a)) for i in a]

[0.03859720653796217,
 0.023798785812803643,
 0.03859720653796217,
 0.023798785812803643,
 0.008531053164706046,
 1.0,
 0.8267725512017572,
 0.9992137797272087,
 0.44794061853226674,
 0.00112378631457686,
 0.00015394682926202823,
 0.0,
 0.00224757262915372,
 0.0020936257998916924,
 0.018309049092180316,
 0.02260536478658118,
 0.02260536478658118,
 0.03506029584816606,
 0.03549431186978377,
 0.059797943562816105,
 0.059797943562816105,
 0.059797943562816105,
 0.03402004939641092,
 0.03402004939641092,
 0.03402004939641092,
 0.018309049092180313,
 0.018309049092180313,
 0.009716417703378586,
 0.009716417703378586,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.11892451741865263,
 0.06986483742499068,
 0.05267957464738723,
 0.03549431186978377,
 0.47945972010769083,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.00112378631457686,
 0.001

In [108]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys, AllChem
from rdkit.Avalon import pyAvalonTools as fpAvalon
from rdkit.Chem.AtomPairs import Pairs, Torsions
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.ChemicalFeatures import BuildFeatureFactory
from rdkit.Chem import rdMolDescriptors
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import sys
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import time

# implemented fingerprints:
# ECFC0 (ecfc0), ECFP0 (ecfp0), MACCS (maccs),
# atom pairs (ap), atom pairs bit vector (apbv), topological torsions (tt)
# hashed atom pairs (hashap), hashed topological torsions (hashtt) --> with 1024 bits
# ECFP4 (ecfp4), ECFP6 (ecfp6), ECFC4 (ecfc4), ECFC6 (ecfc6) --> with 1024 bits
# FCFP4 (fcfp4), FCFP6 (fcfp6), FCFC4 (fcfc4), FCFC6 (fcfc6) --> with 1024 bits
# Avalon (avalon) --> with 1024 bits
# long Avalon (laval) --> with 16384 bits
# long ECFP4 (lecfp4), long ECFP6 (lecfp6), long FCFP4 (lfcfp4), long FCFP6 (lfcfp6) --> with 16384 bits
# RDKit with path length = 5 (rdk5), with path length = 6 (rdk6), with path length = 7 (rdk7)
# 2D pharmacophore (pharm) ?????????????

calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
nbits = 1024
longbits = 16384

# dictionary
fpdict = {}
fpdict['ecfp0'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 0, nBits=nbits)
fpdict['ecfp2'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 1, nBits=nbits)
fpdict['ecfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=nbits)
fpdict['ecfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, nBits=nbits)
#fpdict['ecfc0'] = lambda m: AllChem.GetMorganFingerprint(m, 0)
#fpdict['ecfc2'] = lambda m: AllChem.GetMorganFingerprint(m, 1)
#fpdict['ecfc4'] = lambda m: AllChem.GetMorganFingerprint(m, 2)
#fpdict['ecfc6'] = lambda m: AllChem.GetMorganFingerprint(m, 3)
fpdict['fcfp2'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 1, useFeatures=True, nBits=nbits)
fpdict['fcfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, useFeatures=True, nBits=nbits)
fpdict['fcfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, useFeatures=True, nBits=nbits)
#fpdict['fcfc2'] = lambda m: AllChem.GetMorganFingerprint(m, 1, useFeatures=True)
#fpdict['fcfc4'] = lambda m: AllChem.GetMorganFingerprint(m, 2, useFeatures=True)
#fpdict['fcfc6'] = lambda m: AllChem.GetMorganFingerprint(m, 3, useFeatures=True)
fpdict['lecfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=longbits)
fpdict['lecfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, nBits=longbits)
fpdict['lfcfp4'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 2, useFeatures=True, nBits=longbits)
fpdict['lfcfp6'] = lambda m: AllChem.GetMorganFingerprintAsBitVect(m, 3, useFeatures=True, nBits=longbits)
fpdict['maccs'] = lambda m: MACCSkeys.GenMACCSKeys(m)
#fpdict['ap'] = lambda m: Pairs.GetAtomPairFingerprint(m)
#fpdict['tt'] = lambda m: Torsions.GetTopologicalTorsionFingerprintAsIntVect(m)
fpdict['hashap'] = lambda m: rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(m, nBits=nbits)
fpdict['hashtt'] = lambda m: rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(m, nBits=nbits)
fpdict['avalon'] = lambda m: fpAvalon.GetAvalonFP(m, nbits)
fpdict['laval'] = lambda m: fpAvalon.GetAvalonFP(m, longbits)
fpdict['rdk5'] = lambda m: Chem.RDKFingerprint(m, maxPath=5, fpSize=nbits, nBitsPerHash=2)
fpdict['rdk6'] = lambda m: Chem.RDKFingerprint(m, maxPath=6, fpSize=nbits, nBitsPerHash=2)
fpdict['rdk7'] = lambda m: Chem.RDKFingerprint(m, maxPath=7, fpSize=nbits, nBitsPerHash=2)
fpdict['rdkDes'] = lambda m: calc.CalcDescriptors(m)
long_fps = {'laval', 'lecfp4', 'lecfp6', 'lfcfp4', 'lfcfp6'}

def CalculateFP(fp_name, smiles):
    
    m = Chem.MolFromSmiles(smiles)
    return fpdict[fp_name](m)

#####################################################################################################################
def combine_fp(fp_name):
    
    feat_list = fp_name.split('_')
    not_found = []
    features = []
    descriptors = []
    
    for i in tqdm_notebook(range(len(smiles_list))):
        feat = ''
        ds = ''
        for j in range(len(feat_list)):

            try:
                fp = CalculateFP(feat_list[j], smiles_list[i])
                if feat_list[j] == 'rdkDes':
                    fp = np.asarray(fp)
                    
                    fp = fp.reshape(1,200)

                else:
                    fp = fp.ToBitString()
                    fp = np.array([(np.fromstring(fp,'u1') - ord('0'))], dtype=np.float16)

            except:
#                if feat_list[j] == 'rdkDes':
#                    descriptors.append(np.nan)
#                else:
#                    fingerprints.append(np.nan)
#                    not_found.append(i)
                pass

            if j == 0:
                feat = fp
            
            else:
                feat = np.concatenate((feat, fp), axis=1)
                    
        features.append(feat)
    return features
#        if ds != '':
#            descriptors.append(ds)
#        else:
#
#    not_found = list(set(not_found))

In [129]:
a = combine_fp('rdkDes_lecfp6_laval_hashap')

/Users/pankajverma/anaconda3/envs/seqcomhol/lib/python3.6/site-packages/ipykernel/__main__.py:93: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [130]:
a = np.array(a)

In [44]:
a = np.asarray(a)

In [131]:
a.shape

(1525, 1, 33992)

In [132]:
a = a.reshape(a.shape[0],a.shape[2])

In [133]:
a=np.nan_to_num(a, copy=True, nan=np.nan, posinf=np.nan, neginf=np.nan)

In [134]:
col_mean = np.nanmean(a, axis = 0) 
inds = np.where(np.isnan(a)) 
a[inds] = np.take(col_mean, inds[1]) 

In [135]:
# a = np.vstack(a).astype(np.float64)

# 
scaler = MinMaxScaler()
a = scaler.fit_transform(a)  

In [114]:
print(np.max(a), np.min(a))

nan nan


In [104]:
np.any(np.isnan(a))

True

In [105]:
np.any(np.isfinite(a))

True

array([ 1.68593750e+01, -1.98046875e+00,  1.68593750e+01,  6.75659180e-02,
        6.66992188e-01,  3.92500000e+02,  3.63250000e+02,  3.92250000e+02,
        1.54000000e+02,  0.00000000e+00,  1.89941406e-01, -3.89648438e-01,
        3.89648438e-01,  1.89941406e-01,  1.32128906e+00,  2.00000000e+00,
        2.64257812e+00,  1.83789062e+00,  8.04500000e+02,  2.07031250e+01,
        1.66093750e+01,  1.66093750e+01,  1.29609375e+01,  1.01406250e+01,
        1.01406250e+01,  9.75000000e+00,  9.75000000e+00,  8.78906250e+00,
        8.78906250e+00,  7.26171875e+00,  7.26171875e+00, -1.37011719e+00,
                   inf,  1.99218750e+01,  5.83203125e+00,  2.01367188e+00,
        1.63875000e+02,  1.53203125e+01,  1.22109375e+01,  1.72343750e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  9.58593750e+00,
        4.39062500e+00,  0.00000000e+00,  0.00000000e+00,  2.55000000e+01,
        5.65937500e+01,  1.67500000e+01,  6.10546875e+00,  2.92968750e+01,
        1.15703125e+01,  

In [115]:
list(map(tuple, np.where(np.isinf(a))))

[(411, 411), (10, 12)]

In [125]:
inds = np.where(np.isnan(a))

In [126]:
len(inds)

2

In [127]:
inds

(array([], dtype=int64), array([], dtype=int64))

In [124]:
a[411]

array([ 1.52632040e+01, -4.75760219e+00,  1.52632040e+01,  8.21195993e-03,
        3.44812659e-02,  1.49240600e+03,  1.40571800e+03,  1.49146940e+03,
        5.26000000e+02,  0.00000000e+00,  2.36899494e-01, -3.94137066e-01,
        4.19693163e-01,  3.94137066e-01,  7.11340206e-01,  1.32989691e+00,
        1.96907216e+00,  1.21804146e+00,  3.22106346e+03,  6.99687404e+01,
        5.39176246e+01,  5.88462508e+01,  4.65305589e+01,  3.19874047e+01,
        3.85117483e+01,  2.39966599e+01,  3.02149161e+01,  1.62522708e+01,
        2.25730420e+01,  1.09909453e+01,  1.62068751e+01, -7.24987013e+00,
        1.76678804e+23,  7.71070942e+01,  3.93697293e+01,  2.55281879e+01,
        5.77907562e+02,  5.29539813e+01,  1.81255225e+01,  0.00000000e+00,
        2.36287189e+01,  0.00000000e+00,  2.45104752e+02,  2.89650907e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  7.61850996e+01,
        6.96896090e+01,  3.04719575e+01,  3.65858827e+01,  8.17292306e+01,
        1.21463901e+02,  